In [28]:
import numpy as np
import pandas as pd
import gym
from gym import spaces
from stable_baselines3 import PPO
from joblib import load
import joblib

In [31]:
# Load the trained demand forecasting model
#demand_model = load("demand_forecasting_model.pkl")
#loaded_model = joblib.load("demand_forecasting_model.pkl")

# Load the processed dataset
#df = pd.read_csv("processed_sales_data.csv")

# Predict future demand using the trained model
#X = df.drop(columns=['Date', 'Quantity_Sold'])  # Features
#df['Predicted_Demand'] = demand_model.predict(X)


In [30]:
import pandas as pd
from joblib import load
import os

# Load the processed dataset
df = pd.read_csv("processed_sales_data.csv")

# --- Model Loading with Validation ---
if not os.path.exists("demand_forecasting_model.pkl"):
    raise FileNotFoundError("Model file not found.")

try:
    demand_model = load("demand_forecasting_model.pkl")
except Exception as e:
    raise ValueError(f"Model loading failed: {e}")

if not hasattr(demand_model, "predict"):
    raise ValueError("Invalid model object.")

# --- Predict Demand ---
X = df.drop(columns=['Date', 'Quantity_Sold'])  # Features
try:
    df['Predicted_Demand'] = demand_model.predict(X)
except Exception as e:
    raise ValueError(f"Prediction failed: {e}")

print("Demand prediction successful!")

ValueError: Prediction failed: need to call fit or load_model beforehand

In [2]:
# Define the RL environment
class InventoryEnv(gym.Env):
    def _init_(self, predicted_demand, initial_inventory=100, holding_cost=1, ordering_cost=10, stockout_cost=5):
        super(InventoryEnv, self)._init_()
        
        # Initialize parameters
        self.predicted_demand = predicted_demand
        self.initial_inventory = initial_inventory
        self.holding_cost = holding_cost
        self.ordering_cost = ordering_cost
        self.stockout_cost = stockout_cost
        
        # Define action and observation space
        self.action_space = spaces.Discrete(50)  # Order quantity (0 to 49)
        self.observation_space = spaces.Box(
            low=np.array([0, 0]),  # Inventory level, Predicted demand
            high=np.array([1000, 100]), dtype=np.float32
        )
        
        # Reset the environment
        self.reset()
    
    def reset(self):
        # Reset the state
        self.current_inventory = self.initial_inventory
        self.current_step = 0
        return self._get_state()
    
    def _get_state(self):
        # Get the current state
        return np.array([self.current_inventory, self.predicted_demand[self.current_step]])
    
    def step(self, action):
        # Get the predicted demand for the current step
        demand = self.predicted_demand[self.current_step]
        
        # Calculate the new inventory level
        self.current_inventory = max(0, self.current_inventory - demand) + action
        
        # Calculate costs
        holding_cost = self.holding_cost * self.current_inventory
        ordering_cost = self.ordering_cost * (action > 0)
        stockout_cost = self.stockout_cost * max(0, demand - self.current_inventory)
        
        # Calculate reward
        reward = -(holding_cost + ordering_cost + stockout_cost)
        
        # Move to the next step
        self.current_step += 1
        
        # Check if the episode is done
        done = self.current_step >= len(self.predicted_demand) - 1
        
        # Return the next state, reward, done, and info
        return self._get_state(), reward, done, {}
    
    def render(self, mode='human'):
        # Optional: Render the environment (e.g., print the current state)
        print(f"Step: {self.current_step}, Inventory: {self.current_inventory}, Demand: {self.predicted_demand[self.current_step]}")


^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Create the RL environment
env = InventoryEnv(df['Predicted_Demand'].values)

# Initialize the RL agent (PPO algorithm)
model = PPO('MlpPolicy', env, verbose=1)

# Train the RL agent
model.learn(total_timesteps=10000)

# Save the RL model
model.save("inventory_rl_model")

  Using cached stable_baselines3-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached torch-2.6.0-cp39-cp39-win_amd64.whl.metadata (28 kB)


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\http\client.py", line 462, in read
    n = self.readinto(b)
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\http\client.py", line 506,


  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 1.5/1.5 MB 3.6 MB/s eta 0:00:00
Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
Using cached torch-2.6.0-cp39-cp39-win_amd64.whl (204.1 MB)
   ---------------------------------------- 6.2/6.2 MB 752.6 kB/s eta 0:00:00
   --------------------------               25.7/39.5 MB 482.9 kB/s eta 0:00:29



    n = self.fp.readinto(b)
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_internal\cli\base_command.py", line 106, in _run_wrapper
    status = _inner_run()
  File "C:\Users\Iworld\AppData\Local\Programs\Python\Python39\lib\site-packages\pip\_internal\cli\base_command.py", line 97, in _inner_run
    return self.run(options, args)
  File "C:\

In [ ]:
# Evaluate the RL agent
obs = env.reset()
total_reward = 0
done = False

while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()

print(f"Total Reward: {total_reward}")

In [ ]:
# Extract EOQ and ROP from the RL agent's policy
# EOQ: Average order quantity
# ROP: Inventory level at which the agent decides to reorder
eog = np.mean([model.predict(env._get_state())[0] for _ in range(100)])
rop = np.mean([env._get_state()[0] for _ in range(100)])

print(f"Optimal EOQ: {eog}")
print(f"Optimal ROP: {rop}")